# Loading packages and settings

In [34]:
#installing wetterdienst API package
!pip install wetterdienst

In [35]:
#installing geopandas
#install geopython libraries
!apt install gdal-bin python-gdal python3-gdal
#install python3-rtree - Geopandas requirement
!apt install python3-rtree 
#install geopandas
!pip install git+git://github.com/geopandas/geopandas.git
#install descartes - Geopandas requirement
!pip install descartes

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-d4p97o3n
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-d4p97o3n
  fatal: remote error:
    The unauthenticated git protocol on port 9418 is no longer supported.
  Please see https://github.blog/2021-09-01-improving-git-protocol-security-github/ for more information.
ERROR: Command errored out with exit status 128: git clone -q git

In [36]:
!pip install --upgrade geopandas

!pip install --upgrade pyshp

!pip install --upgrade shapely

!pip install --upgrade descartes

In [37]:
#importing packages and classes from API package

import pandas as pd
from wetterdienst.provider.dwd.observation import DwdObservationRequest
from wetterdienst import Settings
from google.colab import drive
import geopandas as gpd

In [38]:
#Changing settings of wetterdienst

Settings.tidy = True #default, tidy data
Settings.humanize = True #default, humanized parameters
Settings.si_units = False #DON'T convert values to SI units. For original units, see: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/DESCRIPTION_obsgermany_climate_10min_tu_historical_en.pdf

In [39]:
#packages for the projection
from functools import partial
from pyproj import Proj, transform
from shapely.ops import transform
from shapely.geometry import Point
import numpy as np

# Weather databases download

Downloading all datapoints for 3 weather databases: temperature, precipitation and visibility. All data outputs saved at data/raw.


## Temperature

In [ ]:
#air temperature database, 1 hour granularity, all stations
#for list of stations, see: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/zehn_min_tu_Beschreibung_Stationen.txt 

temperature = DwdObservationRequest(
    parameter=["air_temperature"],
    resolution="hourly",
    start_date="2018-01-01",
    end_date="2021-01-01"
).filter_by_station_id(station_id=["0400", "0410","0420", "0430", "0433"]) #Berlin-Buch, Berlin-Kaniswall, Marzahn, Tegel, Tempelhof (Alexanderplatz doesn't have data past 2015. Link: )

Dircache located at /root/.cache/wetterdienst


In [ ]:
temperature.df.head(10) #list of stations

,station_id,from_date,to_date,height,latitude,longitude,name,state
45,00400,1991-01-01 00:00:00+00:00,2022-04-08 00:00:00+00:00,60.0,52.6310,13.5021,Berlin-Buch,Berlin
47,00410,2004-05-01 00:00:00+00:00,2020-06-15 00:00:00+00:00,33.0,52.4040,13.7309,Berlin-Kaniswall,Berlin
48,00420,2007-08-01 00:00:00+00:00,2022-04-08 00:00:00+00:00,61.0,52.5447,13.5598,Berlin-Marzahn,Berlin
51,00430,1986-01-01 00:00:00+00:00,2021-05-05 00:00:00+00:00,36.0,52.5644,13.3088,Berlin-Tegel,Berlin
52,00433,1951-01-01 00:00:00+00:00,2022-04-08 00:00:00+00:00,48.0,52.4675,13.4021,Berlin-Tempelhof,Berlin


In [ ]:
#getting all values
temperature_df = temperature.values.all().df #all values for all stations

KeyboardInterrupt: ignored

In [ ]:
#saving data - temperatures
#drive.mount('/content/drive')
path = '/content/drive/My Drive/temperature.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  temperature_df.to_csv(f)

In [ ]:
#saving data - station info
path = '/content/drive/My Drive/temperature_stations.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  temperature.df.to_csv(f)

## Precipitation

In [ ]:
#precipitation, 10 minutes granularity, all stations
#for list of stations, see: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/zehn_min_tu_Beschreibung_Stationen.txt 

precipitation = DwdObservationRequest(
    parameter=["precipitation"],
    resolution="minute_10",
    start_date="2017-12-31",
    end_date="2021-01-01"
).filter_by_station_id(station_id=["0400", "0410","0420", "0430", "0433"])

In [ ]:
precipitation.df.head()

,station_id,from_date,to_date,height,latitude,longitude,name,state
77,00400,2003-06-10 00:00:00+00:00,2022-04-08 00:00:00+00:00,60.0,52.6310,13.5021,Berlin-Buch,Berlin
78,00410,2003-08-07 00:00:00+00:00,2020-06-15 00:00:00+00:00,33.0,52.4040,13.7309,Berlin-Kaniswall,Berlin
79,00420,2007-08-01 00:00:00+00:00,2022-04-09 00:00:00+00:00,61.0,52.5447,13.5598,Berlin-Marzahn,Berlin
81,00430,1994-12-23 00:00:00+00:00,2021-05-04 00:00:00+00:00,36.0,52.5644,13.3088,Berlin-Tegel,Berlin
82,00433,1995-01-02 00:00:00+00:00,2022-04-08 00:00:00+00:00,48.0,52.4675,13.4021,Berlin-Tempelhof,Berlin


In [ ]:
precipitation_df = precipitation.values.all().df #all values for all stations

parameters {'end_of_interval', 'true_local_time'} are skipped in tidy format as the date parameters are currently not converted to floats
parameters {'end_of_interval', 'true_local_time'} are skipped in tidy format as the date parameters are currently not converted to floats
parameters {'end_of_interval', 'true_local_time'} are skipped in tidy format as the date parameters are currently not converted to floats
parameters {'end_of_interval', 'true_local_time'} are skipped in tidy format as the date parameters are currently not converted to floats
parameters {'end_of_interval', 'true_local_time'} are skipped in tidy format as the date parameters are currently not converted to floats


In [ ]:
precipitation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2369535 entries, 0 to 2369534
Data columns (total 6 columns):
 #   Column      Dtype              
---  ------      -----              
 0   station_id  category           
 1   dataset     category           
 2   parameter   category           
 3   date        datetime64[ns, UTC]
 4   value       float64            
 5   quality     float64            
dtypes: category(3), datetime64[ns, UTC](1), float64(2)
memory usage: 61.0 MB


In [ ]:
#save precipitation data

path = '/content/drive/My Drive/precipitation.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  precipitation_df.to_csv(f)

  #saving data - station info
path = '/content/drive/My Drive/precipitation_stations.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  precipitation.df.to_csv(f)

## Visibility

In [ ]:
#visibility, 1 hour granularity, all stations
#for list of stations, see: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/zehn_min_tu_Beschreibung_Stationen.txt 

visibility = DwdObservationRequest(
    parameter=["visibility"],
    resolution="hourly",
    start_date="2018-01-01",
    end_date="2021-01-01"
).filter_by_station_id(station_id=["0400", "0410","0420", "0430", "0433"]) #Berlin-Buch, Berlin-Kaniswall, Marzahn, Tegel, Tempelhof (Alexanderplatz doesn't have data past 2015. Link: )

In [ ]:
visibility_df = visibility.values.all().df #all values for all stations

parameters {'true_local_time', 'end_of_interval'} are skipped in tidy format as the date parameters are currently not converted to floats
parameters {'true_local_time', 'end_of_interval'} are skipped in tidy format as the date parameters are currently not converted to floats
parameters {'true_local_time', 'end_of_interval'} are skipped in tidy format as the date parameters are currently not converted to floats
parameters {'true_local_time', 'end_of_interval'} are skipped in tidy format as the date parameters are currently not converted to floats
parameters {'true_local_time', 'end_of_interval'} are skipped in tidy format as the date parameters are currently not converted to floats


In [ ]:
visibility_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131525 entries, 0 to 131524
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   station_id  131525 non-null  category           
 1   dataset     131525 non-null  category           
 2   parameter   131525 non-null  category           
 3   date        131525 non-null  datetime64[ns, UTC]
 4   value       26300 non-null   float64            
 5   quality     26300 non-null   float64            
dtypes: category(3), datetime64[ns, UTC](1), float64(2)
memory usage: 3.4 MB


In [ ]:
visibility_df.groupby(by=["parameter", "station_id"]).count() #only station 430 has data

dataset   date  value  quality
parameter  station_id                                
visibility 00400         26305  26305      0        0
           00410         26305  26305      0        0
           00420         26305  26305      0        0
           00430         26305  26305  26300    26300
           00433         26305  26305      0        0

In [ ]:
#save visibility data
path = '/content/drive/My Drive/data/raw/visibility.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  visibility_df.to_csv(f)

  #saving data - station info
path = '/content/drive/My Drive/data/raw/visibility_stations.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  visibility.df.to_csv(f)

# Data recodes and checks

## Temperature

In [ ]:
#loading data
drive.mount("/content/drive")
path = '/content/drive/My Drive/data/raw/temperature.csv'
temperature_df = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
temperature_df.rename(columns={ temperature_df.columns[0]: "col_id" }, inplace=True)
temperature_df.set_index("col_id", inplace=True)


In [ ]:
temperature_df['year'] = pd.DatetimeIndex(temperature_df['date']).year
temperature_df['month'] = pd.DatetimeIndex(temperature_df['date']).month
temperature_df['day'] = pd.DatetimeIndex(temperature_df['date']).day
temperature_df['hour'] = pd.DatetimeIndex(temperature_df['date']).hour
temperature_df = temperature_df[temperature_df["year"] != 2021]

In [ ]:
temperature_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263040 entries, 0 to 263048
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   station_id  263040 non-null  int64  
 1   dataset     263040 non-null  object 
 2   parameter   263040 non-null  object 
 3   date        263040 non-null  object 
 4   value       231341 non-null  float64
 5   quality     231341 non-null  float64
 6   year        263040 non-null  int64  
 7   month       263040 non-null  int64  
 8   day         263040 non-null  int64  
 9   hour        263040 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 22.1+ MB


In [ ]:
temperature_df.head(10)

,station_id,dataset,parameter,date,value,quality,year,month,day,hour
col_id,,,,,,,,,,
0,400,temperature_air,temperature_air_mean_200,2018-01-01 00:00:00+00:00,11.8,3.0,2018,1,1,0
1,400,temperature_air,temperature_air_mean_200,2018-01-01 01:00:00+00:00,11.8,3.0,2018,1,1,1
2,400,temperature_air,temperature_air_mean_200,2018-01-01 02:00:00+00:00,11.6,3.0,2018,1,1,2
3,400,temperature_air,temperature_air_mean_200,2018-01-01 03:00:00+00:00,11.5,3.0,2018,1,1,3
4,400,temperature_air,temperature_air_mean_200,2018-01-01 04:00:00+00:00,10.7,3.0,2018,1,1,4
5,400,temperature_air,temperature_air_mean_200,2018-01-01 05:00:00+00:00,10.0,3.0,2018,1,1,5
6,400,temperature_air,temperature_air_mean_200,2018-01-01 06:00:00+00:00,10.0,3.0,2018,1,1,6
7,400,temperature_air,temperature_air_mean_200,2018-01-01 07:00:00+00:00,9.6,3.0,2018,1,1,7
8,400,temperature_air,temperature_air_mean_200,2018-01-01 08:00:00+00:00,8.9,3.0,2018,1,1,8


In [ ]:
temperature_df.groupby(by=["station_id", "year"]).count() #number of non-missing data per station and year

dataset  parameter   date  value  quality  month    day  \
station_id year                                                            
400        2018    17520      17520  17520  17520    17520  17520  17520   
           2019    17520      17520  17520  17520    17520  17520  17520   
           2020    17568      17568  17568  17568    17568  17568  17568   
410        2018    17520      17520  17520  16468    16468  17520  17520   
           2019    17520      17520  17520      0        0  17520  17520   
           2020    17568      17568  17568   5006     5006  17568  17568   
420        2018    17520      17520  17520  17520    17520  17520  17520   
           2019    17520      17520  17520  17520    17520  17520  17520   
           2020    17568      17568  17568  17568    17568  17568  17568   
430        2018    17520      17520  17520  17520    17520  17520  17520   
           2019    17520      17520  17520  17520    17520  17520  17520   
           2020    17568      17568  17568  17568    17568  17568  17568   
433        2018    17520      17520  17520  17230    17230  17520  17520   
           2019    17520      17520  17520  17245    17245  17520  17520   
           2020    17568      17568  17568  17568    17568  17568  17568   

                  hour  
station_id year         
400        2018  17520  
           2019  17520  
           2020  17568  
410        2018  17520  
           2019  17520  
           2020  17568  
420        2018  17520  
           2019  17520  
           2020  17568  
430        2018  17520  
           2019  17520  
           2020  17568  
433        2018  17520  
           2019  17520  
           2020  17568

In [ ]:
temperature_df.groupby(by=["parameter"]).count() #values per parameter, hourly has less parameters? mjust humidity and temperature_air_mean_200

,station_id,dataset,date,value,quality,year,month,day,hour
parameter,,,,,,,,,
humidity,131520,131520,131520,115558,115558,131520,131520,131520,131520
temperature_air_mean_200,131520,131520,131520,115783,115783,131520,131520,131520,131520


In [ ]:
temperature_df.groupby(by=["month", "parameter"]).mean() #checking that means make sense

station_id      value  quality         year  \
month parameter                                                               
1     humidity                       418.6  83.232837      3.0  2019.000000   
      temperature_air_mean_200       418.6   3.205180      3.0  2019.000000   
2     humidity                       418.6  73.787138      3.0  2019.011765   
      temperature_air_mean_200       418.6   2.988191      3.0  2019.011765   
3     humidity                       418.6  70.687886      3.0  2019.000000   
      temperature_air_mean_200       418.6   4.657003      3.0  2019.000000   
4     humidity                       418.6  57.962599      3.0  2019.000000   
      temperature_air_mean_200       418.6  11.921706      3.0  2019.000000   
5     humidity                       418.6  61.193217      3.0  2019.000000   
      temperature_air_mean_200       418.6  14.547715      3.0  2019.000000   
6     humidity                       418.6  60.628982      3.0  2019.000000   
      temperature_air_mean_200       418.6  20.150443      3.0  2019.000000   
7     humidity                       418.6  61.778743      3.0  2019.000000   
      temperature_air_mean_200       418.6  20.094479      3.0  2019.000000   
8     humidity                       418.6  59.387821      3.0  2019.000000   
      temperature_air_mean_200       418.6  21.442566      3.0  2019.000000   
9     humidity                       418.6  68.437259      3.0  2019.000000   
      temperature_air_mean_200       418.6  15.872254      3.0  2019.000000   
10    humidity                       418.6  79.397126      3.0  2019.000000   
      temperature_air_mean_200       418.6  11.245813      3.0  2019.000000   
11    humidity                       418.6  85.613889      3.0  2019.000000   
      temperature_air_mean_200       418.6   6.250780      3.0  2019.000000   
12    humidity                       418.6  85.104527      3.0  2019.000000   
      temperature_air_mean_200       418.6   4.209611      3.0  2019.000000   

                                      day  hour  
month parameter                                  
1     humidity                  16.000000  11.5  
      temperature_air_mean_200  16.000000  11.5  
2     humidity                  14.670588  11.5  
      temperature_air_mean_200  14.670588  11.5  
3     humidity                  16.000000  11.5  
      temperature_air_mean_200  16.000000  11.5  
4     humidity                  15.500000  11.5  
      temperature_air_mean_200  15.500000  11.5  
5     humidity                  16.000000  11.5  
      temperature_air_mean_200  16.000000  11.5  
6     humidity                  15.500000  11.5  
      temperature_air_mean_200  15.500000  11.5  
7     humidity                  16.000000  11.5  
      temperature_air_mean_200  16.000000  11.5  
8     humidity                  16.000000  11.5  
      temperature_air_mean_200  16.000000  11.5  
9     humidity                  15.500000  11.5  
      temperature_air_mean_200  15.500000  11.5  
10    humidity                  16.000000  11.5  
      temperature_air_mean_200  16.000000  11.5  
11    humidity                  15.500000  11.5  
      temperature_air_mean_200  15.500000  11.5  
12    humidity                  16.000000  11.5  
      temperature_air_mean_200  16.000000  11.5

## Precipitation

In [84]:
#loading data
#drive.mount("/content/drive")
path = '/content/drive/My Drive/data/raw/precipitation.csv'
precipitation_df = pd.read_csv(path)

In [86]:
precipitation = clean_cols(precipitation_df)

In [ ]:
precipitation.groupby(by=["year", "month", "parameter"]).mean()

station_id     value  quality   day  \
year month parameter                                                         
2018 1     precipitation_duration           418.6  1.881832      3.0  16.0   
           precipitation_height             418.6  0.015235      3.0  16.0   
           precipitation_indicator_wr       418.6  0.242608      3.0  16.0   
     2     precipitation_duration           418.6  0.435640      3.0  14.5   
           precipitation_height             418.6  0.000786      3.0  14.5   
...                                           ...       ...      ...   ...   
2020 11    precipitation_height             418.6  0.003998      3.0  15.5   
           precipitation_indicator_wr       418.6  0.069097      3.0  15.5   
     12    precipitation_duration           418.6  0.734543      3.0  16.0   
           precipitation_height             418.6  0.004704      3.0  16.0   
           precipitation_indicator_wr       418.6  0.104279      3.0  16.0   

                                       hour  
year month parameter                         
2018 1     precipitation_duration      11.5  
           precipitation_height        11.5  
           precipitation_indicator_wr  11.5  
     2     precipitation_duration      11.5  
           precipitation_height        11.5  
...                                     ...  
2020 11    precipitation_height        11.5  
           precipitation_indicator_wr  11.5  
     12    precipitation_duration      11.5  
           precipitation_height        11.5  
           precipitation_indicator_wr  11.5  

[108 rows x 5 columns]

In [ ]:
precipitation.groupby(by=["station_id", "parameter"]).count()

dataset    date   value  quality  \
station_id parameter                                                      
400        precipitation_duration       157968  157968       0        0   
           precipitation_height         157968  157968  157968   157968   
           precipitation_indicator_wr   157968  157968       0        0   
410        precipitation_duration       157968  157968       0        0   
           precipitation_height         157968  157968   64607    64607   
           precipitation_indicator_wr   157968  157968       0        0   
420        precipitation_duration       157968  157968       0        0   
           precipitation_height         157968  157968  157968   157968   
           precipitation_indicator_wr   157968  157968       0        0   
430        precipitation_duration       157968  157968  157968   157968   
           precipitation_height         157968  157968  157447   157447   
           precipitation_indicator_wr   157968  157968  157968   157968   
433        precipitation_duration       157968  157968  157646   157646   
           precipitation_height         157968  157968  157594   157594   
           precipitation_indicator_wr   157968  157968  157646   157646   

                                         year   month     day    hour  
station_id parameter                                                   
400        precipitation_duration      157968  157968  157968  157968  
           precipitation_height        157968  157968  157968  157968  
           precipitation_indicator_wr  157968  157968  157968  157968  
410        precipitation_duration      157968  157968  157968  157968  
           precipitation_height        157968  157968  157968  157968  
           precipitation_indicator_wr  157968  157968  157968  157968  
420        precipitation_duration      157968  157968  157968  157968  
           precipitation_height        157968  157968  157968  157968  
           precipitation_indicator_wr  157968  157968  157968  157968  
430        precipitation_duration      157968  157968  157968  157968  
           precipitation_height        157968  157968  157968  157968  
           precipitation_indicator_wr  157968  157968  157968  157968  
433        precipitation_duration      157968  157968  157968  157968  
           precipitation_height        157968  157968  157968  157968  
           precipitation_indicator_wr  157968  157968  157968  157968

In [ ]:
precipitation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2369520 entries, 0 to 2369533
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   station_id  int64  
 1   dataset     object 
 2   parameter   object 
 3   date        object 
 4   value       float64
 5   quality     float64
 6   year        int64  
 7   month       int64  
 8   day         int64  
 9   hour        int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 198.9+ MB


## Visibility

In [40]:
#load data
path = '/content/drive/My Drive/data/raw/visibility.csv'
visibility = pd.read_csv(path)

In [64]:
visibility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131525 entries, 0 to 131524
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  131525 non-null  int64  
 1   station_id  131525 non-null  int64  
 2   dataset     131525 non-null  object 
 3   parameter   131525 non-null  object 
 4   date        131525 non-null  object 
 5   value       26300 non-null   float64
 6   quality     26300 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 7.0+ MB


In [65]:
visibility.head()

,Unnamed: 0,station_id,dataset,parameter,date,value,quality
0,0,400,visibility,visibility,2018-01-01 00:00:00+00:00,NaN,NaN
1,1,400,visibility,visibility,2018-01-01 01:00:00+00:00,NaN,NaN
2,2,400,visibility,visibility,2018-01-01 02:00:00+00:00,NaN,NaN
3,3,400,visibility,visibility,2018-01-01 03:00:00+00:00,NaN,NaN
4,4,400,visibility,visibility,2018-01-01 04:00:00+00:00,NaN,NaN


In [ ]:
#checking the number of data points
vis = visibility.copy()
vis["year"] = pd.DatetimeIndex(vis["date"]).year

vis[vis["station_id"] == 430].groupby(by = vis["year"]).count()

,Unnamed: 0,station_id,dataset,parameter,date,value,quality,year
year,,,,,,,,
2018,8760,8760,8760,8760,8760,8760,8760,8760
2019,8760,8760,8760,8760,8760,8760,8760,8760
2020,8784,8784,8784,8784,8784,8779,8779,8784
2021,1,1,1,1,1,1,1,1


## Collision data and weather stations

In [41]:
#collision data
#drive.mount("/content/drive")
path = '/content/drive/MyDrive/data/output/collisions_shp.dbf'
collision = gpd.read_file(path)

In [ ]:
#weather stations - temperature dataset
path = '/content/drive/My Drive/data/raw/temperature_stations.csv'
stations = pd.read_csv(path)

In [81]:
#weather stations - precipitation dataset
path = '/content/drive/My Drive/data/raw/precipitation_stations.csv'
stations_precipitation = pd.read_csv(path)

In [ ]:
#weather stations - visibility dataset
path = "/content/drive/My_Drive/data/raw/visibility_stations.csv"
stations_visibility = pd.read_csv(path)

# Data transformations and data extraction

## Functions that are applied to all dataframes

Getting distances from data points to weather stations and preparing weather databases for merge 

In [42]:
def clean_cols(df):
  df.rename(columns={df.columns[0]: "col_id" }, inplace=True)
  df.set_index("col_id", inplace=True)
  #recoding date to match colision dataframe
  df['year'] = pd.DatetimeIndex(df['date']).year
  df['month'] = pd.DatetimeIndex(df['date']).month
  df['day'] = pd.DatetimeIndex(df['date']).day
  df['hour'] = pd.DatetimeIndex(df['date']).hour
  df = df[df["year"] != 2021] #filter 2021 out
  return df

In [43]:
def get_distances(collision, stations):

  #takes a clean collision geodataframe and stations csv dataframe, returns a dataframe with distances for every collision in collisions dataset

  #turn stations dataframe to GeoDataframe
  stations = gpd.GeoDataFrame(
    stations, geometry=gpd.points_from_xy(stations.longitude, stations.latitude)) #we just need to define where geometry points are saved
  stations["geometry"] = stations["geometry"].set_crs("EPSG:4326") #set the geometry to latitude/longitude
  
  #set projection to calculate distance in meters. points need to already be set to EPSG:4326 #CHECK WITH 3174
  collision = collision.to_crs(epsg=3149)
  stations = stations.to_crs(epsg=3149)

  #dictionary from stations geodataframe
  points = stations.set_index("station_id").to_dict()["geometry"] #weather stations into dictionary

  #calculate distance
  distance = []
  for key, value in points.items(): #iterrating over dictionary pairs
    d = collision.distance(value).rename(key) #calculating distance from collision dataframe and saving in columns named by the key
    distance.append(d)
  
  return pd.concat(distance, axis=1)

In [44]:
def clean_weather(df):

  #setting data types
  data = df.astype("object")

  #creating unique indetifier
  data["datetime"] = pd.to_datetime(data[['year', 'month', 'day', "hour"]])


  #pivoting
  d = data.pivot_table('value', ["datetime", "date", "station_id"], 'parameter') #pivoting temperature dataframe, to get weather data
  d = d.reset_index()

  d["station_id"] = d["station_id"].astype("object") #so that data types are the same for later merge


  return d

## Temperature

In [ ]:
def get_temperature(collision, distances, temperature):

  #takes collision dataframe, distances dataframe and pivoted temperature dataframe and outputs temperature and humidity with col index from collision

  collision_d = collision
  collision_d["st_closest"] = distances_full.loc[:, "400":"433"].idxmin(axis=1).astype("object") #geting the station_id of the nearest station
  collision_d["st_2closest"] = distances_full.loc[:, "400":"433"].mask(distances_full.loc[:, "400":"433"].eq(distances_full.min(axis=1), axis=0)).idxmin(axis=1).astype("object") #getting second closest by masking the closest

  #setting datetime for unique identification in collision dataframe
  collision_d = collision_d.rename(columns={"weekday": "day"})
  collision_d["datetime"] = pd.to_datetime(collision_d[['year', 'month', 'day', "hour"]])

  #getting data for the closest and second closest station
  collision_d = collision_d.merge(temperature[["temperature_air_mean_200", "humidity", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_closest"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  collision_d = collision_d.merge(temperature[["temperature_air_mean_200", "humidity", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_2closest"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  
  #filling missing data for the closest station with data from the second closest station
  collision_d["temperature_air_mean_200_x"] = collision_d["temperature_air_mean_200_x"].fillna(collision_d["temperature_air_mean_200_y"])
  collision_d["humidity_x"] = collision_d["humidity_x"].fillna(collision_d["humidity_y"])

  #renaming columns
  collision_d = collision_d.rename(columns={"temperature_air_mean_200_x": "temperature", "humidity_x": "humidity"})

  return collision_d[["col_id", "objectid", "temperature", "humidity"]]



In [ ]:
#for temperature extraction
distances_full = get_distances(collision, stations)

In [ ]:
#pivoted dataframe for temperature
pivot = clean_weather(temperature_df)

In [ ]:
collision_temperature = get_temperature(collision, distances_full, pivot)

In [ ]:
collision_temperature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38851 entries, 0 to 38850
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   col_id       38851 non-null  int64  
 1   objectid     38851 non-null  object 
 2   temperature  38851 non-null  float64
 3   humidity     38851 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.5+ MB


In [ ]:
#saving data - temperatures
#drive.mount('/content/drive')
path = '/content/drive/My Drive/data/output/collision_temperature.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  collision_temperature.to_csv(f)

## Precipitation

In [103]:
precipitation = clean_cols(precipitation_df)

In [104]:
#distances for precipitation
distances_prec = get_distances(collision, stations_precipitation)

In [ ]:
distances_prec.head()

,400,410,420,430,433
0,16510.840982,26628.036638,8408.130192,15792.831520,8932.539592
1,18891.801271,45314.167261,23566.146806,3521.675757,19169.190398
2,16171.666559,31438.564615,12111.369442,10893.880458,8306.709636
3,24537.139456,34376.585874,20003.483226,11981.401498,4998.623842
4,10675.426083,37464.139664,14343.960655,8438.908809,16158.899622


In [87]:
#pivoted dataframe for precipitation
pivot_prec = clean_weather(precipitation)

KeyboardInterrupt: ignored

In [ ]:
#saving pivot as it takes long
path = '/content/drive/My Drive/precipitation_pivot.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  pivot_prec.to_csv(f)

In [105]:
#opening pivot from drive

path = '/content/drive/My Drive/precipitation_pivot.csv'
pivot_prec = pd.read_csv(path)

In [106]:
#rolling average for precipitation
pivot_prec_av = pivot_prec.copy()

pivot_prec_av["prec_duration"] = pivot_prec.groupby('station_id')['precipitation_duration'].rolling(10, min_periods=6).mean().reset_index(0,drop=True)
pivot_prec_av["prec_height"] = pivot_prec.groupby('station_id')['precipitation_height'].rolling(10, min_periods=6).mean().reset_index(0,drop=True)

#filtering out all data that is not full hour
pivot_prec_av["date"] = pd.to_datetime(pivot_prec_av['date'])
pivot_prec_av["datetime"] = pd.to_datetime(pivot_prec_av['datetime'])
pivot_prec_av = pivot_prec_av.loc[pivot_prec_av["date"].dt.minute == 0]
pivot_prec_av = pivot_prec_av.drop(["precipitation_indicator_wr", "precipitation_duration", "precipitation_height"], axis=1)

In [107]:
pivot_prec_av.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116026 entries, 0 to 696136
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   Unnamed: 0     116026 non-null  int64              
 1   datetime       116026 non-null  datetime64[ns]     
 2   date           116026 non-null  datetime64[ns, UTC]
 3   station_id     116026 non-null  int64              
 4   prec_duration  52601 non-null   float64            
 5   prec_height    115925 non-null  float64            
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(2), int64(2)
memory usage: 6.2 MB


In [194]:
filtered = pivot_prec_av["station_id"] >= 430
filtered_df = pivot_prec_av[filtered]
#filtered_df.info()
bool_index = filtered_df["prec_duration"].isnull()
filtered_df[bool_index]

,Unnamed: 0,datetime,date,station_id,prec_duration,prec_height
3,3,2017-12-31,2017-12-31 00:00:00+00:00,430,NaN,NaN
4,4,2017-12-31,2017-12-31 00:00:00+00:00,433,NaN,NaN


In [243]:
def get_precipitation(collision, distances, precipitation):

  #takes collision dataframe, distances dataframe and pivoted precipitation dataframe with rolling averages and outputs precipitation hight and duration with col index from collision
  
  collision_d = collision

  #gets closest station for precipitation duration (only two stations with data: 430 & 433)
  collision_d["st_closest_duration"] = distances.loc[:, "430":"433"].idxmin(axis=1).astype("object")
  collision_d["st_2closest_duration"] = distances.loc[:, "430":"433"].mask(distances.loc[:, "430":"433"].eq(distances.min(axis=1), axis=0)).idxmin(axis=1).astype("object")
  
  #gets closest station for precipitation hight
  collision_d["st_closest_height"] = distances.loc[:, "400":"433"].idxmin(axis=1).astype("object") #geting the station_id of the nearest station
  collision_d["st_2closest_height"] = distances.loc[:, "400":"433"].mask(distances.loc[:, "400":"433"].eq(distances.min(axis=1), axis=0)).idxmin(axis=1).astype("object") #getting second closest by masking the closest

  #setting datetime for unique identification in collision dataframe
  collision_d = collision_d.rename(columns={"weekday": "day"})
  collision_d["datetime"] = pd.to_datetime(collision_d[['year', 'month', 'day', "hour"]])
  #collision_d["hour_before"] = collision_d["datetime"] - pd.Timedelta(1, unit="h")

  #getting precipitation duration
  collision_d = collision_d.merge(precipitation[["prec_duration", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_closest_duration"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  collision_d = collision_d.merge(precipitation[["prec_duration", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_2closest_duration"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])

  #getting precipitation height for the closest and second closest station
  collision_d = collision_d.merge(precipitation[["prec_height", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_closest_height"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  collision_d = collision_d.merge(precipitation[["prec_height", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_2closest_height"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  
  #filling missing data for precipitation duration and height for the closest station with data from the second closest station
  collision_d["prec_duration_x"] = collision_d["prec_duration_x"].fillna(collision_d["prec_duration_y"])
  collision_d["prec_duration_x"] = collision_d["prec_duration_x"].fillna(collision_d["prec_duration_x"].mean()) #fill the rest of the missing with the mean
  collision_d["prec_height_x"] = collision_d["prec_height_x"].fillna(collision_d["prec_height_y"])
  
  #renaming columns
  collision_d = collision_d.rename(columns={"prec_duration_x":"prec_duration","prec_height_x": "prec_height"})

  return collision_d[["col_id", "objectid", "prec_duration", "prec_height"]]

In [244]:
collision_precipitation = get_precipitation(collision, distances_prec, pivot_prec_av)

In [202]:
###filtered = collision_precipitation["station_id"] >= 430
###filtered_df = pivot_prec_av[filtered]
###filtered_df.info()
##bool_index = collision_precipitation["prec_duration"].isnull()
##collision_precipitation[bool_index]

,col_id,objectid,prec_duration,prec_height,datetime
23906,23906,199557,NaN,0.0,2019-09-07 21:00:00
23915,23915,199566,NaN,0.0,2019-09-07 21:00:00
23945,23945,199596,NaN,0.0,2019-09-07 20:00:00


In [204]:
#pivot_prec_av[pivot_prec_av["datetime"] == "2019-09-07 21:00:00"]

,Unnamed: 0,datetime,date,station_id,prec_duration,prec_height
404050,404050,2019-09-07 21:00:00,2019-09-07 21:00:00+00:00,400,NaN,0.000
404051,404051,2019-09-07 21:00:00,2019-09-07 21:00:00+00:00,420,NaN,0.000
404052,404052,2019-09-07 21:00:00,2019-09-07 21:00:00+00:00,430,9.7,0.005


In [205]:
#pivot_prec_av[pivot_prec_av["datetime"] == "2019-09-07 20:00:00"]

,Unnamed: 0,datetime,date,station_id,prec_duration,prec_height
404032,404032,2019-09-07 20:00:00,2019-09-07 20:00:00+00:00,400,NaN,0.000
404033,404033,2019-09-07 20:00:00,2019-09-07 20:00:00+00:00,420,NaN,0.000
404034,404034,2019-09-07 20:00:00,2019-09-07 20:00:00+00:00,430,7.2,0.001


In [245]:
collision_precipitation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38851 entries, 0 to 38850
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   col_id         38851 non-null  int64  
 1   objectid       38851 non-null  object 
 2   prec_duration  38851 non-null  float64
 3   prec_height    38851 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.5+ MB


In [246]:
collision_precipitation.head()

,col_id,objectid,prec_duration,prec_height
0,0,112695,0.0,0.000
1,1,112705,1.1,0.000
2,2,112726,10.0,0.519
3,3,112737,2.1,0.039
4,4,112747,0.0,0.000


In [247]:
path = '/content/drive/My Drive/data/output/collision_precipitation.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  collision_precipitation.to_csv(f)

## Visibility

In [45]:
#cleaning columns in visibility dataframe
visibility_filtered = visibility[visibility["station_id"] == 430].copy() #only for station 430 that has data
visibility_filtered = clean_cols(visibility_filtered)

In [46]:
#pivoted visibility data
pivot_visibility = clean_weather(visibility_filtered)

In [16]:
pivot_visibility.head()

parameter,datetime,date,station_id,visibility
0,2018-01-01 00:00:00,2018-01-01 00:00:00+00:00,430,18000.0
1,2018-01-01 01:00:00,2018-01-01 01:00:00+00:00,430,40000.0
2,2018-01-01 02:00:00,2018-01-01 02:00:00+00:00,430,50000.0
3,2018-01-01 03:00:00,2018-01-01 03:00:00+00:00,430,50000.0
4,2018-01-01 04:00:00,2018-01-01 04:00:00+00:00,430,30000.0


In [75]:
def get_visibility(collision, visibility):

  #takes collision dataframe and pivoted visibility dataframe and outputs visibility with col index from collision
  #we don't need distances from weather station as there is only one station 

  collision_d = collision

  #setting datetime for unique identification in collision dataframe
  collision_d = collision_d.rename(columns={"weekday": "day"})
  collision_d["datetime"] = pd.to_datetime(collision_d[['year', 'month', 'day', "hour"]])

  #getting visibility
  collision_d["hour_before"] = collision_d["datetime"] - pd.Timedelta(1, unit="h") #setting a column for an hour before to fill missing data

  collision_d = collision_d.merge(visibility[["visibility", "datetime"]], how = "left", left_on="datetime", right_on="datetime").drop(columns = ["datetime"]) #getting data for the hour of the collision
  collision_d = collision_d.merge(visibility[["visibility", "datetime"]], how = "left", left_on="hour_before", right_on="datetime").drop(columns = ["datetime"]) #getting data for hour before

  collision_d["visibility_x"] = collision_d["visibility_x"].fillna(collision_x["visibility_y"]) #filling missing data

  collision_d = collision_d.rename(columns={"visibility_x":"visibility"})

  return collision_d[["col_id", "objectid", "visibility"]]


In [76]:
collision_visibility = get_visibility(collision, pivot_visibility)

In [78]:
collision_visibility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38851 entries, 0 to 38850
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   col_id      38851 non-null  int64  
 1   objectid    38851 non-null  object 
 2   visibility  38851 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ MB


In [79]:
#saving data
path = '/content/drive/My Drive/data/output/collision_visibility.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  collision_visibility.to_csv(f)

In [ ]:
#geopy gives correct distance, unilike geopandas
dist = geopy.distance.geodesic((13.50210, 52.63100), (13.42725, 52.63382)) #Buch to accident 0
print(dist)

8.28662021473542 km
